In [7]:
import boto3 
import sagemaker
from ibov.utils import load_config
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel
import os
import io
import boto3
import numpy as np
import pandas as pd

### Loading Configs

In [8]:
# Load config dict
config = load_config()

In [9]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

In [10]:
dropout = config.get("model").get("dropout")
window = config.get("feature").get("window")
hidden_layer = config.get("model").get("hidden_layer")
lr = config.get("model").get("lr")
seed = config.get("model").get("seed")
epochs = config.get("model").get("epochs")

### Upload Data

In [11]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [12]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [25]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge',
                    hyperparameters = {
                        "batch-size": 50,
                        "epochs": epochs,
                        "seed": seed,
                        "input-layer": window,
                        "hidden-layer": hidden_layer,
                        "dropout": dropout        
                    })

In [26]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-17 02:36:26 Starting - Starting the training job...
2021-01-17 02:36:48 Starting - Launching requested ML instancesProfilerReport-1610850984: InProgress
......
2021-01-17 02:37:56 Starting - Preparing the instances for training.........
2021-01-17 02:39:37 Downloading - Downloading input data...
2021-01-17 02:40:21 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-17 02:40:22,516 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-17 02:40:22,543 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-17 02:40:22,547 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-17 02:40:22,861 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-01-17 02:40:22,861 sagemaker-containers INFO     G

02:40:43, epoch: 8, train: 0.499, valid: 0.038
02:40:46, epoch: 9, train: 0.476, valid: 0.041
02:40:50, epoch: 10, train: 0.424, valid: 0.045
02:40:54, epoch: 11, train: 0.345, valid: 0.033
02:40:58, epoch: 12, train: 0.309, valid: 0.028
02:41:03, epoch: 13, train: 0.286, valid: 0.033
02:41:08, epoch: 14, train: 0.215, valid: 0.022
02:41:14, epoch: 15, train: 0.232, valid: 0.045
02:41:19, epoch: 16, train: 0.321, valid: 0.053
02:41:25, epoch: 17, train: 0.313, valid: 0.102
02:41:31, epoch: 18, train: 0.312, valid: 0.025
02:41:38, epoch: 19, train: 0.28, valid: 0.032
02:41:45, epoch: 20, train: 0.262, valid: 0.074
02:41:52, epoch: 21, train: 0.257, valid: 0.024
02:41:59, epoch: 22, train: 0.243, valid: 0.028
02:42:08, epoch: 23, train: 0.237, valid: 0.06
02:42:17, epoch: 24, train: 0.218, valid: 0.022
2021-01-17 02:42:17,728 sagemaker-containers INFO     Reporting training SUCCESS

2021-01-17 02:42:33 Uploading - Uploading generated training model
2021-01-17 02:42:33 Completed - Trainin

In [27]:
estimator.latest_training_job.job_name

'sagemaker-pytorch-2021-01-17-02-36-21-851'

In [17]:
estimator.model_data

's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-17-00-45-10-011/output/model.tar.gz'

In [120]:
oi = "s3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-19-20-39-951/output/model.tar.gz"

In [121]:
model = PyTorchModel(model_data=oi,#estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     py_version="py3", 
                     entry_point='server.py',
                     source_dir='ibov')

In [122]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------!

In [123]:
client = boto3.client("sagemaker-runtime")

In [124]:
ibovespa = pd.read_csv(os.path.join("data", "data.csv"))

In [125]:
with open(os.path.join("data", "data.csv")) as f:
    saddd = f.read().encode('utf-8')

In [127]:
respose = client.invoke_endpoint(EndpointName='sagemaker-pytorch-2021-01-16-21-23-45-663',
                       Body=saddd,
                       ContentType='text/csv',
                       Accept='Accept')

In [128]:
resposta = respose["Body"].read().decode("utf-8")

In [132]:
import ast

In [133]:
ast.literal_eval(resposta)

[0.10484547168016434,
 0.2182859480381012,
 0.3236715495586395,
 0.4115184545516968,
 0.4766795337200165,
 0.5254829525947571,
 0.5601202249526978,
 0.5821685791015625,
 0.5982837080955505,
 0.6081803441047668,
 0.6124605536460876,
 0.6142750978469849,
 0.6132918000221252,
 0.607500433921814,
 0.6003987193107605,
 0.5944706797599792,
 0.5902760624885559,
 0.5876886248588562,
 0.5857018232345581,
 0.58204185962677,
 0.5782886147499084,
 0.5774833559989929,
 0.5808299779891968,
 0.5858021378517151,
 0.5915355682373047,
 0.5967764854431152,
 0.6023174524307251,
 0.6091407537460327,
 0.6158046722412109,
 0.6216511726379395,
 0.6281389594078064,
 0.6340134143829346,
 0.6387987732887268,
 0.6433814764022827,
 0.6474251747131348,
 0.6501380205154419,
 0.6528744101524353,
 0.6562739610671997,
 0.6589218378067017,
 0.6605839729309082,
 0.6614805459976196,
 0.6599161028862,
 0.6578516960144043,
 0.6577481627464294,
 0.6554138660430908,
 0.6506258249282837,
 0.6461991667747498,
 0.641720235347747